In [ ]:
import os
import requests
from pydub import AudioSegment

# Convert the file to wav
audio = AudioSegment.from_file("Sample_Audio.m4a", format="m4a")
audio.export("Sample_Audio.wav", format="wav")

# Path to the converted audio file
audio_path = "Sample_Audio.wav"

# AssemblyAI API key
api_key = "Enter Your API Key" #Assembply api is free

# Upload the audio file to AssemblyAI
def upload_file(filename):
    headers = {
        'authorization': api_key
    }
    response = requests.post(
        'https://api.assemblyai.com/v2/upload',
        headers=headers,
        data=open(filename, 'rb')
    )
    return response.json()

# Request a transcription
def request_transcription(audio_url):
    endpoint = "https://api.assemblyai.com/v2/transcript"
    json = {
        "audio_url": audio_url
    }
    headers = {
        "authorization": api_key,
        "content-type": "application/json"
    }
    response = requests.post(endpoint, json=json, headers=headers)
    return response.json()

# Get the transcription result
def get_transcription(transcription_id):
    endpoint = f"https://api.assemblyai.com/v2/transcript/{transcription_id}"
    headers = {
        "authorization": api_key
    }
    response = requests.get(endpoint, headers=headers)
    return response.json()

# Upload the file
upload_response = upload_file(audio_path)
audio_url = upload_response['upload_url']

# Request the transcription
transcription_response = request_transcription(audio_url)
transcription_id = transcription_response['id']

# Wait for the transcription to complete
import time
while True:
    transcription_result = get_transcription(transcription_id)
    if transcription_result['status'] == 'completed':
        transcription_text = transcription_result['text']
        print("Transcription: ", transcription_text)
        # Save the transcription to a file
        with open("transcription.txt", "w") as file:
            file.write(transcription_text)
        break
    elif transcription_result['status'] == 'failed':
        print("Transcription failed")
        break
    time.sleep(5)

# Clean up: remove the converted wav file
os.remove(audio_path)
